In [1]:
import cv2
import tensorflow as tf
import numpy as np
from keras import models, layers, callbacks
from keras.preprocessing import image
import os
from prettytable import PrettyTable

2023-01-08 21:32:18.555728: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-08 21:32:18.747128: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/mike/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:
2023-01-08 21:32:18.747152: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-08 21:32:19.810216: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: 

In [2]:
model= models.load_model('ModelSug2Group1.h5')

2023-01-08 21:32:22.665406: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/mike/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:
2023-01-08 21:32:22.665437: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-08 21:32:22.665461: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mike-VirtualBox): /proc/driver/nvidia/version does not exist
2023-01-08 21:32:22.665742: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate comp

In [3]:
def color_segment_function(img_array):
    img_array= np.rint(img_array)
    img_array= img_array.astype('uint8')
    hsv_img= cv2.cvtColor(img_array, cv2.COLOR_RGB2HSV)
    mask = cv2.inRange(hsv_img, (24, 50, 0), (55, 255, 255))
    result = cv2.bitwise_and(img_array, img_array, mask=mask)
    result= result.astype('float64')
    return result


In [4]:
#image function from keras.preprocessing
train_datagen = image.ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.0,
      height_shift_range=0.0,
      shear_range=0.0,
      zoom_range=0.0,
      horizontal_flip=True,
      vertical_flip= True,
    preprocessing_function=color_segment_function,
      fill_mode='nearest')

test_datagen = image.ImageDataGenerator(rescale=1./255, preprocessing_function=color_segment_function)

In [7]:
path ='/home/mike/Downloads'
test_ = os.path.join(path, 'Competition_group1')
test = test_datagen.flow_from_directory(
        test_,
        target_size=(150,150),
        batch_size=1,
        class_mode='categorical',
        shuffle=False)

Found 900 images belonging to 1 classes.


In [8]:
pred= model.predict(test, steps= test.n, verbose=1)

900/900 [==============================] - 18s 20ms/step


In [9]:
label_map = {0: 'weed',
 1: 'sugar_beet'}

In [13]:
predicted_class_indices=np.argmax(pred,axis=1)

prediction_labels = [label_map[k] for k in predicted_class_indices]
filenames= test.filenames

In [14]:
import csv
csvfile= open('group1detections.csv', 'w', newline='')
writer= csv.writer(csvfile)

headers= ['id', 'label']

writer.writerow(headers)
t = PrettyTable(headers)
for i, f, p in zip(range(len(filenames)), filenames, prediction_labels):
    writer.writerow([os.path.basename(f),p])
    if i <10:
        t.add_row([os.path.basename(f), p])
    elif i<13:
        t.add_row(['.', '.'])
csvfile.close()
print(t)

+---------+-------+
|    id   | label |
+---------+-------+
| 000.png |  weed |
| 001.png |  weed |
| 002.png |  weed |
| 003.png |  weed |
| 004.png |  weed |
| 005.png |  weed |
| 006.png |  weed |
| 007.png |  weed |
| 008.png |  weed |
| 009.png |  weed |
|    .    |   .   |
|    .    |   .   |
|    .    |   .   |
+---------+-------+


In [16]:
file = open('results.txt', 'x')
file.write('id;label\n')
for i in range (len(prediction_labels)):
    if(i<10):
        file.write('00'+str(i)+';'+prediction_labels[i]+'\n')
    elif(1<100):
        file.write('0'+str(i)+ ';'+prediction_labels[i]+'\n')
    else:
        file.write(str(i)+';'+prediction_labels[i]+'\n')